In [3]:
import numpy as np
import pandas as pd
from pyDOE2 import *

# Recursos
- [Catapulta](https://sigmazone.com/catapult/)
- [Documentação pyDOE2](https://pythonhosted.org/pyDOE/)
- [Tutorial de DOE com catapulta do Sigmazone](https://sigmazone.com/quantum_xl_doe_tutorial/#Step_1:_Setting_up_the_Designed_Experiment)
- [Response Optimization with Design of Experiments and python](https://towardsdatascience.com/response-optimization-with-design-of-experiments-and-python-63f9afb3f26f)
- [Youtube: Design of Experiments (DOE) - Minitab Masters Module 5](https://www.youtube.com/watch?v=rXmrMu-czM4)
- [Curso de Design of Experiments da PennState](https://online.stat.psu.edu/stat503/lesson/1)

## Realizando o experimento.
Colocamos as configurações de cada linha do dataframe na [catapulta](https://sigmazone.com/catapult/) para preencher a coluna `distance` e salvá-lo como um novo dataframe.

A planilha compartilhada está disponível no Google Sheets: [LHS](https://docs.google.com/spreadsheets/d/1e-EHZcRrpu5jy7baX_Z1_XDp0cppIyoXXveRIzQKk2E/edit#gid=1081657349) e [Box-Benhken](https://docs.google.com/spreadsheets/d/1gL6UifW3i-ZbZff8haHjBg8py-GPBVFXIqHzISFiOiQ/edit#gid=1572290238)

In [26]:
df1 = pd.read_csv("../df_catapulta_complete.csv")
# df1 = df1.dropna(ignore_index=True)
df1 = df1.dropna().drop('Unnamed: 0', axis=1)
df1 = df1.reset_index(drop=True)
df1['distance'] = df1['distance'].astype(int)
df1.shape

(48, 6)

In [25]:
df2 = pd.read_csv("../df_box_complete.csv")
df2 = df2.dropna().drop('Unnamed: 0', axis=1)
df2 = df2.astype(int)
df2.shape

(41, 6)

In [16]:
df = pd.concat([df1, df2])
df

,release_angle,firing_angle,cup_elevation,bungee_position,pin_elevation,distance
0,183,100,211,168,120,185
1,173,124,240,110,100,125
2,128,104,243,117,163,96
3,148,131,273,135,138,82
4,123,90,265,194,118,105
...,...,...,...,...,...,...
36,170,106,250,100,100,143
37,170,106,250,200,100,229
38,170,106,250,100,200,232
39,170,106,250,200,200,380


## Análise dos dados
Usamos a biblioteca `pyDOE2` para realizar a análise de variância, entre outras.

## Otimização
Técnica de Análise de Superfície de Resposta (RSM) para achar a melhor configuração da catapulta.

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.optimize import minimize

In [28]:
# Variáveis independentes
x = df[['release_angle', 'firing_angle','cup_elevation', 'bungee_position', 'pin_elevation']]

# Variável de resposta
y = df['distance']

# Divide dataset em variáveis de treinamento e de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

Ao utilizar a técnica de análise de superfície de resposta (RSM), transformar as variáveis de teste em um polinômio permite capturar relações não lineares entre os fatores e a resposta. É uma forma de expandir o espaço de recursos e levar em consideração possíveis interações e efeitos não lineares entre as variáveis.

In [29]:
poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)

In [30]:
# Treina o modelo
model = LinearRegression()
model.fit(x_train_poly, y_train)

# Predições
y_pred = model.predict(x_test_poly)

In [33]:
# Avalia o desempenho do modelo. Quanto menor o MSE, melhor.
mse = mean_squared_error(y_test, y_pred, squared=False)
mse

21.91453205152406